In [1]:
import pandas as pd

In [2]:
import json

In [6]:
from pandas.io.json import json_normalize
data = pd.read_json('userA_trips.json',  lines=True) 
print(data)


     end_location_lat  end_location_lon  \
0            37.35340        -121.95782   
1            37.39951        -121.97882   
2            37.38881        -121.98369   
3            37.39855        -121.97594   
4            37.35346        -121.95822   
5            37.35252        -121.95808   
6            37.35315        -121.95531   
7            37.34109        -121.97354   
8            37.39845        -121.97649   
9            37.35323        -121.95760   
10           37.39884        -121.97678   
11           37.35351        -121.95775   
12           37.35355        -121.95780   
13           37.35161        -122.00594   
14           37.35277        -122.00063   
15           37.35345        -121.95818   
16           37.35203        -121.96804   
17           37.35314        -121.97291   
18           37.35364        -121.95812   
19           37.35326        -121.97281   
20           37.35169        -121.96039   
21           37.35191        -122.00399   
22         

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 11 columns):
end_location_lat       684 non-null float64
end_location_lon       684 non-null float64
end_location_name      684 non-null object
end_time               684 non-null datetime64[ns]
end_timezone           684 non-null object
id                     684 non-null object
start_location_lat     684 non-null float64
start_location_lon     684 non-null float64
start_location_name    684 non-null object
start_time             684 non-null datetime64[ns]
start_timezone         684 non-null object
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 58.9+ KB


In [8]:
data['end_location_name'].nunique()

163

In [9]:
data['end_location_lon'].nunique()

472

In [10]:
data['end_location_lat'].nunique()

376

In [11]:
data['start_location_name'].nunique()

177